<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/image_classification_tribeau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
# drive.mount('/content/drive')

In [129]:
# 環境変数読み込み
READ_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/accumulated_list_tribeau_eye_until_500.xlsx'
ROOT_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/症例'

In [130]:
# データ読み込み

# Excelファイルのパス
file_path = READ_FILE_PATH
df = pd.read_excel(file_path)
df.head()

,クリニック名,クリニック住所,URL,ドクター名,メニュー名,費用,副作用・リスク,患者属性,施術_1,施術_2,...,after1Month_3,after0Day_1,after0Day_2,after0Day_3,after0Day_4,after0Day_5,after0Day_6,after0Day_7,after0Day_8,after0Day_9
0,ルラ美容クリニック 東京駅前院,東京都中央区日本橋2-1-17 丹生ビル4階,https://tribeau.jp/case_reports/32354,山本 泰輔,ルラループ二重術,"￥126,500～",【ダウンタイム】 \n腫れ：大きな腫れ2-3日 \n浮腫んだような腫れ1-2週間 \n内出...,10代女性,二重埋没 / 目元,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LOUIS CLINIC 二子玉川院【ルイクリニック】,東京都世田谷区玉川3-23-18 ザ・シティ二子玉川4階,https://tribeau.jp/case_reports/31422,小松 塁,タレ目形成（埋没）+目尻切開,通常価格220000円→特別価格65000円,感染\n左右差\n痛み\nむくみ\n赤み\n腫れ\n内出血,30代女性,目尻切開 / 目元,タレ目形成・グラマラスライン（埋没） / 目元,...,NaN,"https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...","https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...","https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...",NaN,NaN,NaN,NaN,NaN,NaN
2,9BeautyClinic,東京都中央区銀座1-9-13 プライム銀座柳通りビル7階,https://tribeau.jp/case_reports/32350,池田 陸,クマ取り,"98,000円",腫れ：数日～1週間程度\n内出血：稀に内出血が生じ、まぶたが赤紫色になる場合がある\n ...,20代女性,クマ治療 / 目元,下まぶたたるみ取り / 目元,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9BeautyClinic,東京都中央区銀座1-9-13 プライム銀座柳通りビル7階,https://tribeau.jp/case_reports/32349,池田 陸,二重埋没・クマ取り,"39,800円～",腫れ：数日～1週間程度\n内出血：稀に内出血が生じ、まぶたが赤紫色になる場合がある\n ...,20代女性,二重埋没 / 目元,クマ治療 / 目元,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NEXUS CLINIC,東京都千代田区有楽町2-3-5 aune有楽町6F,https://tribeau.jp/case_reports/32348,NaN,ネクサスアイズ(埋没法),"¥268,000(ﾓﾆﾀｰ価格・税込・麻酔代込)",腫れ/内出血/違和感/ゴロゴロ感/痛み/左右差/ラインの不整/ラインの消失/感染/頭痛,女性,二重埋没 / 目元,NaN,...,NaN,"https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
# カラム名編集

# 変換表生成
conversion_dict = {
    "二重埋没 / 目元": "二重/埋没法, Mb",
    "目頭切開 / 目元": "目頭切開, MGs",
    "ヒアルロン酸注入（涙袋） / 目元": "涙袋形成, Nb",
    "クマ治療 / 目元": "クマ取り, Km",
    "その他（目元） / 目元": "その他, Ot",
    "眉下切開 / 目元": "眉下切開, MSk",
    "下まぶたの脂肪取り / 目元": "脂肪取り, Sb",
    "二重切開 / 目元": "二重/目頭切開, Sk",
    "タレ目形成・グラマラスライン（切開） / 目元": "タレ目/目頭切開, Sk",
    "目元修正 / 目元": "その他, Ot",
    "目尻切開 / 目元": "その他, Ot",
    "眼瞼下垂（切開） / 目元": "眼瞼下垂, Gn",
    "脂肪注入（目の下） / 目元": "涙袋形成, Nb",
    "下まぶたたるみ取り / 目元": "シワ取り, Sw",
    "上まぶたたるみ取り / 目元": "シワ取り, Sw",
    "自然癒着法 / 目元": "その他, Ot",
    "ボトックス（タレ目） / 目元": "ボトックス注入, Bt",
    "ヒアルロン酸注入（目の下） / 目元": "シワ取り, Sw",
    "ボトックス（眉間） / 目元": "ボトックス注入, Bt",
    "蒙古ひだ形成 / 目元": "その他, Ot",
    "タレ目形成・グラマラスライン（埋没） / 目元": "タレ目/埋没法, TRMb",
    "前額リフト / 目元": "シワ取り, Sw",
    "眼瞼下垂（非切開） / 目元": "眼瞼下垂, Gn",
    "目尻靭帯移動 / 目元": "その他, Ot",
    "ボトックス（目尻） / 目元": "ボトックス注入, Bt",
    "上まぶたの脂肪取り / 目元": "脂肪取り, Sb",
    "その他 / その他": "その他, Ot"
}

# カラム名のリストから "施術_" で始まるカラムをフィルタリング
treatment_columns = [col for col in df.columns if col.startswith("施術_")]

# "施術_" で始まるカラム内からconversion_dictに一致するキーを値に変換
# 一致しないキーがあればNaNに置き換える
for col in treatment_columns:
  df[col] = df[col].apply(lambda x: conversion_dict.get(x, np.nan))

# 結果確認
df.head(10)

,クリニック名,クリニック住所,URL,ドクター名,メニュー名,費用,副作用・リスク,患者属性,施術_1,施術_2,...,after1Month_3,after0Day_1,after0Day_2,after0Day_3,after0Day_4,after0Day_5,after0Day_6,after0Day_7,after0Day_8,after0Day_9
0,ルラ美容クリニック 東京駅前院,東京都中央区日本橋2-1-17 丹生ビル4階,https://tribeau.jp/case_reports/32354,山本 泰輔,ルラループ二重術,"￥126,500～",【ダウンタイム】 \n腫れ：大きな腫れ2-3日 \n浮腫んだような腫れ1-2週間 \n内出...,10代女性,"二重/埋没法, Mb",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LOUIS CLINIC 二子玉川院【ルイクリニック】,東京都世田谷区玉川3-23-18 ザ・シティ二子玉川4階,https://tribeau.jp/case_reports/31422,小松 塁,タレ目形成（埋没）+目尻切開,通常価格220000円→特別価格65000円,感染\n左右差\n痛み\nむくみ\n赤み\n腫れ\n内出血,30代女性,"その他, Ot","タレ目/埋没法, TRMb",...,NaN,"https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...","https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...","https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...",NaN,NaN,NaN,NaN,NaN,NaN
2,9BeautyClinic,東京都中央区銀座1-9-13 プライム銀座柳通りビル7階,https://tribeau.jp/case_reports/32350,池田 陸,クマ取り,"98,000円",腫れ：数日～1週間程度\n内出血：稀に内出血が生じ、まぶたが赤紫色になる場合がある\n ...,20代女性,"クマ取り, Km","シワ取り, Sw",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9BeautyClinic,東京都中央区銀座1-9-13 プライム銀座柳通りビル7階,https://tribeau.jp/case_reports/32349,池田 陸,二重埋没・クマ取り,"39,800円～",腫れ：数日～1週間程度\n内出血：稀に内出血が生じ、まぶたが赤紫色になる場合がある\n ...,20代女性,"二重/埋没法, Mb","クマ取り, Km",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NEXUS CLINIC,東京都千代田区有楽町2-3-5 aune有楽町6F,https://tribeau.jp/case_reports/32348,NaN,ネクサスアイズ(埋没法),"¥268,000(ﾓﾆﾀｰ価格・税込・麻酔代込)",腫れ/内出血/違和感/ゴロゴロ感/痛み/左右差/ラインの不整/ラインの消失/感染/頭痛,女性,"二重/埋没法, Mb",NaN,...,NaN,"https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,銀座TAクリニック,東京都中央区銀座2丁目3番1号 RayGinza 5階,https://tribeau.jp/case_reports/32347,古川 悠次郎,埋没法✖️クマ取り,"【下眼瞼脱脂術(クマ・目の下のふくらみとり)】通常価格 両目：110,000円",腫れ、痛み、内出血等,女性,"二重/埋没法, Mb","クマ取り, Km",...,NaN,"https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,KAUNIS CLINIC（カウニスクリニック）,"東京都港区南青山5-2-12 Gビル南青山03 1F,2F",https://tribeau.jp/case_reports/32344,NaN,埋没6点固定,"￥198,000 (税込)",腫れ、内出血、瞼の違和感、二重の消失、左右差、ドライアイ,女性,"二重/埋没法, Mb",NaN,...,NaN,"https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,聖心美容クリニック 大阪院,大阪府大阪市北区梅田3-3-45 マルイト西梅田ビル2F,https://tribeau.jp/case_reports/32339,大井 弘一,【30代女性・眼力のあるバランスの良い二重にしたい】二重マイクロメソッド埋没法（3点）＋目頭...,"478,500円（税込）",<二重術（埋没法）：マイクロメソッド+α>\n術後の腫れ、痛み、内出血、左右差、後戻り、シス...,30代女性,"二重/埋没法, Mb","目頭切開, MGs",...,NaN,"https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,秋葉原美容クリニック,東京都千代田区神田佐久間町1-21 鳥山ビル3F,https://tribeau.jp/case_reports/32334,竹村 一人,目頭切開法,143000,組織を取りすぎたり垂れ目形成まで狙って大きく切れ込みを入れすぎると、内側のピンク色ぽい粘膜が...,20代女性,"目頭切開, MGs",NaN,...,NaN,"https://p1-1f54eb9e.imageflux.jp/w=300,h=300,a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LIBER CLINIC,埼玉県さいたま市大宮区仲町2-9 仲町シロタビルEAST302,https://tribeau.jp/case_reports/31454,矢橋 洋一郎,直後からアイメイクも可能！切らない二重もやっぱりやはし！,モニター価格168000円,腫れ、内出血など,女性,"二重/埋没法, Mb",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
def unique_list(input_list):
    seen = set()
    output = []
    for item in input_list:
        if item not in seen:
            seen.add(item)
            output.append(item)

    return output

def filtered_list(input_list):
  input_list_copy = deepcopy(input_list)
  # "二重"と"タレ目"の施術が二つ行われていた場合
  # "タレ目/" を "タレ目(" に変換する処理
  for item in input_list_copy:
        if "二重" in item:  # "二重"が文字列の一部として存在するかを確認
            input_list_copy = [treatment.replace("タレ目/", "タレ目(") for treatment in input_list_copy]
            break

  return input_list_copy

# "施術_"で始まるカラムを選択
treatment_columns = df.filter(like="施術_").columns

# それらのカラムを結合。NaNは空文字列に変換
df['combined'] = df[treatment_columns].apply(lambda row: ''.join([str(x) + ', ' if pd.notna(x) else '' for x in row]), axis=1).str.rstrip(', ')

def format_treatment_value(value):
    treatments = value.split(", ")
    folder_name = []
    tag = []
    # 空の文字列をスキップ
    if treatments:
      # 偶数インデックスの要素をfolder_nameに、奇数インデックスの要素をtagに分ける
      folder_name = unique_list(treatments[::2])
      tag = unique_list(treatments[1::2])

      # データ前処理
      filtered_folder_name = filtered_list(folder_name)

    return ('-'.join(filtered_folder_name), ''.join(tag))

# combinedカラムに関数を適用してtagカラムを生成
df['folder_name'], df['tag'] = zip(*df['combined'].apply(format_treatment_value))

# combinedカラムを削除
df.drop('combined', axis=1, inplace=True)

In [133]:
pd.DataFrame(df['folder_name']).head(100)

,folder_name
0,二重/埋没法
1,その他-タレ目/埋没法
2,クマ取り-シワ取り-脂肪取り
3,二重/埋没法-クマ取り-シワ取り-脂肪取り-涙袋形成
4,二重/埋没法
...,...
95,涙袋形成
96,涙袋形成
97,クマ取り-シワ取り-脂肪取り
98,目頭切開


In [134]:
# ディレクトリの存在を確認し、存在しない場合はディレクトリを作成
directory = os.path.dirname(ROOT_FILE_PATH)
if not os.path.exists(directory):
    os.makedirs(directory)



# DataFrameをエクセルファイルとして保存
df.to_excel(ROOT_FILE_PATH, index=False)

ValueError: ignored

In [ ]:
# :黒い四角_小:画像ネーミンングルール ※命名規則キャメルケースに沿っています
# emEye(No.)(施術名)(ビフォー)(アフター)(経過期間)
# 例：emEye2KmBeAf0D　→ No.2 クマ取り ビフォー アフター施術直後
# 例：emEye3SkBeAf1M　→No.3 目頭切開 ビフォー アフター1ヶ月


施術_1,	施術_2
"二重/埋没法, Mb","クマ取り, Km"

In [ ]:
施術_1,	施術_2
1 "二重/埋没法, Mb","クマ取り, Km"
2 "二重/埋没法, Mb"